In [ ]:

for i in range(3):
    var = "Wave-U-Net-Pytorch/audio_examples/Cristina Vane - So Easy/mix.mp3"
    !python3 Wave-U-Net-Pytorch/predict.py --load_model Wave-U-Net-Pytorch/checkpoints/waveunet/model --input --output

In [31]:
import os
import glob

def get_audio_files_wav_u_net(current_dir):
    # Get the current directory
    current_dir = current_dir

    # Define the filenames to search for
    filenames = ['other.wav', 'vocals.wav', 'bass.wav', 'drums.wav']
    separated_filenames = ['mixture.wav_other.wav', 'mixture.wav_vocals.wav', 'mixture.wav_bass.wav', 'mixture.wav_drums.wav']

    # List to store file paths
    file_paths = []

    # Iterate over each filename
    for filename in filenames:
        # Use glob to search for the file in the current directory
        files_found = glob.glob(os.path.join(current_dir, filename))
        # Add the found file paths to the list
        file_paths.extend(files_found)

    # Display the list of file paths
    #print("File paths found in the current directory:")
    for path in file_paths:
        pass
        #print(path)

    # Now, search in current_dir/mdx/mixture directory
    mixture_dir = os.path.join(current_dir, 'separated', 'wav-u-net')
    #print(f"mix directory : {mixture_dir}")

    # List to store file paths from mixture directory
    mixture_file_paths = []

    # Iterate over each filename
    for filename in separated_filenames:
        # Use glob to search for the file in the mixture directory
        files_found = glob.glob(os.path.join(mixture_dir, filename))
        # Add the found file paths to the list
        mixture_file_paths.extend(files_found)

    # Display the list of file paths from the mixture directory
    #print("\nFile paths found in the mixture directory:")
    for path in mixture_file_paths:
        pass
        #print(path)

    return file_paths, mixture_file_paths
get_audio_files_wav_u_net("/Users/kaimikkelsen/Downloads/test/Al James - Schoolboy Facination")

(['/Users/kaimikkelsen/Downloads/test/Al James - Schoolboy Facination/other.wav',
  '/Users/kaimikkelsen/Downloads/test/Al James - Schoolboy Facination/vocals.wav',
  '/Users/kaimikkelsen/Downloads/test/Al James - Schoolboy Facination/bass.wav',
  '/Users/kaimikkelsen/Downloads/test/Al James - Schoolboy Facination/drums.wav'],
 ['/Users/kaimikkelsen/Downloads/test/Al James - Schoolboy Facination/separated/wav-u-net/mixture.wav_other.wav',
  '/Users/kaimikkelsen/Downloads/test/Al James - Schoolboy Facination/separated/wav-u-net/mixture.wav_vocals.wav',
  '/Users/kaimikkelsen/Downloads/test/Al James - Schoolboy Facination/separated/wav-u-net/mixture.wav_bass.wav',
  '/Users/kaimikkelsen/Downloads/test/Al James - Schoolboy Facination/separated/wav-u-net/mixture.wav_drums.wav'])

In [33]:
import os
import subprocess

# Define the directory where you want to start traversing
root_dir = "/Users/kaimikkelsen/Downloads/test"

# Define the path to the Wave-U-Net model
model_path = "Wave-U-Net-Pytorch/checkpoints/waveunet/model"

#end_run = False
# Iterate through each directory
for dirpath, dirnames, filenames in os.walk(root_dir):
    # Check if mixture.wav exists in the current directory
    if "separated" in dirnames and "wav-u-net" in os.listdir(os.path.join(dirpath, "separated")):
        print(f"Skipping directory {dirpath} as separated/wav-u-net already exists.")
        continue
    
    if "mixture.wav" in filenames:
        print(dirpath)
        # Get the path to mixture.wav
        mixture_path = os.path.join(dirpath, "mixture.wav")
        
        # Create the output directory if it doesn't exist
        output_dir = os.path.join(dirpath, "separated", "wav-u-net")

        print(output_dir)
        os.makedirs(output_dir, exist_ok=True)
        
        # Define the command to run
        command = [
            "python3",
            "Wave-U-Net-Pytorch/predict.py",
            "--load_model",
            model_path,
            "--input",
            mixture_path,
            "--output",
            output_dir
        ]
        
        # Run the command
        subprocess.run(command)
        #break


Skipping directory /Users/kaimikkelsen/Downloads/test/Moosmusic - Big Dummy Shake as separated/wav-u-net already exists.
Skipping directory /Users/kaimikkelsen/Downloads/test/The Mountaineering Club - Mallory as separated/wav-u-net already exists.
Skipping directory /Users/kaimikkelsen/Downloads/test/Bobby Nobody - Stitch Up as separated/wav-u-net already exists.
Skipping directory /Users/kaimikkelsen/Downloads/test/Punkdisco - Oral Hygiene as separated/wav-u-net already exists.
Skipping directory /Users/kaimikkelsen/Downloads/test/Lyndsey Ollard - Catching Up as separated/wav-u-net already exists.
Skipping directory /Users/kaimikkelsen/Downloads/test/Al James - Schoolboy Facination as separated/wav-u-net already exists.
Skipping directory /Users/kaimikkelsen/Downloads/test/James Elder & Mark M Thompson - The English Actor as separated/wav-u-net already exists.
Skipping directory /Users/kaimikkelsen/Downloads/test/The Easton Ellises - Falcon 69 as separated/wav-u-net already exists.
Sk

In [34]:

import os
import csv
import librosa
import mir_eval
import librosa
import numpy as np
import os
import librosa
import argparse
import numpy as np

def getSeparationMetrics(audio_ref_list, audio_list):
    reference_sources = np.array(audio_ref_list)
    estimated_sources = np.array(audio_list)
    #print(reference_sources)
    #print(estimated_sources)
    (sdr, sir, sar, perm) = mir_eval.separation.bss_eval_sources(reference_sources, estimated_sources, False)
    return np.mean(sdr), np.mean(sir), np.mean(sar)

def write_to_csv(data, file_path):
    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Directory", "Other", "Vocals", "Bass", "Drums"])
        for row in data:
            writer.writerow(row)


# Specify the directory path
#directory_path = '/path/to/your/directory'
directory_path = "/Users/kaimikkelsen/Downloads/test"
# Get a list of all directories in the specified directory
directories = [d for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d))]

has_run = False
data_to_write = []
# Loop through each directory
counter = 0
for directory in directories:
    #print(directory)
    reference_files, separated_files = get_audio_files(directory_path+"/"+directory)
    print(directory)
    print(f"{counter} of {len(directories)}")
    counter = counter + 1
    
    if(reference_files and separated_files and not has_run):

        row_data = [directory]
        for i in range(4):

            print(reference_files[i])
            print(separated_files[i])

            audio_ref, _ = librosa.load(reference_files[i])
            audio_sep, _ = librosa.load(separated_files[i])

            ref_list = [audio_ref]
            sep_list = [audio_sep]

            print(ref_list)

            sdr, sir, sar = getSeparationMetrics(ref_list, sep_list)
            print("SDR:", sdr)
            row_data.append(sdr)
            # print("SIR:", sir)
            # print("SAR:", sar)
        data_to_write.append(row_data)
        #has_run = True
        

csv_file_path = "/Users/kaimikkelsen/sdr_eval/wav-u-net-pytorch_output.csv"
write_to_csv(data_to_write, csv_file_path)
print("CSV file written successfully.")
    

        
    # You can perform operations on each directory here




File paths found in the current directory:
mix directory : /Users/kaimikkelsen/Downloads/test/Moosmusic - Big Dummy Shake/separated/wav-u-net

File paths found in the mixture directory:
Moosmusic - Big Dummy Shake
0 of 50
/Users/kaimikkelsen/Downloads/test/Moosmusic - Big Dummy Shake/other.wav
/Users/kaimikkelsen/Downloads/test/Moosmusic - Big Dummy Shake/separated/wav-u-net/mixture.wav_other.wav
[array([ 0., -0.,  0., ...,  0.,  0.,  0.], dtype=float32)]
SDR: 1.642506342004604
/Users/kaimikkelsen/Downloads/test/Moosmusic - Big Dummy Shake/vocals.wav
/Users/kaimikkelsen/Downloads/test/Moosmusic - Big Dummy Shake/separated/wav-u-net/mixture.wav_vocals.wav
[array([ 0., -0.,  0., ...,  0.,  0.,  0.], dtype=float32)]
SDR: 5.857681046750226
/Users/kaimikkelsen/Downloads/test/Moosmusic - Big Dummy Shake/bass.wav
/Users/kaimikkelsen/Downloads/test/Moosmusic - Big Dummy Shake/separated/wav-u-net/mixture.wav_bass.wav
[array([ 0., -0.,  0., ...,  0.,  0.,  0.], dtype=float32)]
SDR: 0.61325768017